# Wearing the Inside Out: Using LSTMNs and Wearable Data to Identify Human Emotion

This project is a test bed using the data from: Sébastien Faye, Nicolas Louveton, Sasan Jafarnejad, Roman Kryvchenko, Thomas Engel. An Open Dataset for Human Activity Analysis using Smart Devices. 2017.

First step is to import what we are going to use.

In [1]:
import pandas as pd
import datetime
import csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import ast
from sklearn import preprocessing
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


A dictionary, *activity_classes*, is used to classify the mutliple classes of activities in the data set into **8** classes.

Then I am manually creating the vector representation of the classes in *activities* dictionary.

*MAX_DIFFERENCE* is the maximum difference in time to consider two events happening at the same time. The difference I have chosen is .05 seconds.

*ZERO_TIME* is just timedelta of 0 to be able to compare difference in times.

In [2]:
activity_classes = {
    "fitness": ["Running", "Train", "Football"],
    "entertainment": ["Video games", "In computer", "Watching TV", "Movie"],
    "transportation": ["In vehicle", "In bus", "On bus stop", "Train"],
    "recreation": ["At home", "Picnic", "Walk", "Cooking", "Pause", "Shop", "Shopping& wearing", "Walking&party"],
    "sleep": ["Sleep"],
    "eat": ["Eat"],
    "work": ["Meeting", "Work"],
    "miscellaneous": ["Phone was out of the pocket (forgot)", "Took off glasses"]
}

activities = {
    "fitness": [1, 0, 0, 0, 0, 0, 0, 0],
    "entertainment": [0, 1, 0, 0, 0, 0, 0, 0],
    "transportation": [0, 0, 1, 0, 0, 0, 0, 0],
    "recreation": [0, 0, 0, 1, 0, 0, 0, 0],
    "sleep": [0, 0, 0, 0, 1, 0, 0, 0],
    "eat": [0, 0, 0, 0, 0, 1, 0, 0],
    "work": [0, 0, 0, 0, 0, 0, 1, 0],
    "miscellaneous": [0, 0, 0, 0, 0, 0, 0, 1]
}

MAX_DIFFERENCE = datetime.timedelta(0,0,500000)
ZERO_TIME = datetime.timedelta(0,0)



Class activity keeps track of the starting and ending time as well as the name of the activity. It also provides useful methods to work with the activities.

In [3]:
class Activity():
    def __init__(self, name=None, start=None, end=None):
        if name is not None:
            for n, values in activity_classes.items():
                if name in values:
                    self.name = n
            # print(self.name)
        else:
            self.name = None
        if start is not None:
            self.start = get_time(start)
        else:
            self.start = None
        if end is not None:
            self.end = get_time(end)
        else:
            self.end = None

    def set_name(self, name):
        for n, values in activity_classes.items():
            if name in values:
                self.name = n
        self.fake_name = name

    def set_start_end(self, start, end):
        self.start = get_time(start)
        self.end = get_time(end)

    def get_class_number(self):
        return activities[self.name]

    def in_between(self, date_time):
        return self.start <= date_time <= self.end

In [4]:
def make_data():
    with open("human-activity-smart-devices/glasses.csv") as g:
        glasses_reader = csv.reader(g)
        with open("human-activity-smart-devices/smartwatch.csv") as w:
            watch_reader = csv.reader(w)
            lines = []
            glasses_rows = []
            glasses_columns = []
            watch_rows = []
            watch_columns = []
            for i, row in enumerate(glasses_reader):
                if i == 0:
                    glasses_columns = row
                else:
                    glasses_rows.append(row)
            for i, row in enumerate(watch_reader):
                if i == 0:
                    watch_columns = row
                else:
                    watch_rows.append(row)

            iter_glasses = iter(glasses_rows)
            iter_watches = iter(watch_rows)
            row_glass = next(iter_glasses)
            row_watch = next(iter_watches)

            write = False
            stop_glass = False
            stop_watch = False
            counter = 0
            while True:
                if row_watch[1] == "heart_rate":
                    if stop_watch and stop_glass:
                        break

                    delta = get_time(row_glass[1]) - get_time(row_watch[2])
                    if abs(delta) < MAX_DIFFERENCE:

                        if delta < ZERO_TIME:
                            lines.append([get_time(row_glass[1])]+ row_glass[3:] + [ast.literal_eval(row_watch[3])[0]])
                        else:
                            lines.append([get_time(row_watch[2])] + row_glass[3:] + [ast.literal_eval(row_watch[3])[0]])
                        counter += 1
                        row_watch = next(iter_watches)
                    elif delta < ZERO_TIME:
                        if write:
                            lines.append([get_time(row_watch[2])]+ ["-" for i in range(10)] + [ast.literal_eval(row_watch[3])[0]])
                            counter += 1
                            write = False
                        else:
                            write = True

                        try:
                            if not stop_glass:
                                row_glass = next(iter_glasses)
                        except StopIteration:
                            stop_glass = True
                    elif delta > ZERO_TIME:
                        if write:
                            lines.append([get_time(row_glass[1])]+ row_glass[3:]+["-"])
                            counter += 1
                            write = False
                        else:
                            write = True

                        try:
                            if not stop_watch:
                                row_watch = next(iter_watches)
                        except StopIteration:
                            stop_watch = True
                    else:

                        raise KeyboardInterrupt()
                else:
                    if stop_watch and stop_glass:
                        break
                    try:
                        if not stop_watch:
                            row_watch = next(iter_watches)
                    except StopIteration:
                        stop_watch = True
                    try:
                        if not stop_glass:
                            row_glass = next(iter_glasses)
                    except StopIteration:
                        stop_glass = True
            columns = ['datetime'] + glasses_columns[3:] + ['heart_beat']
            return columns, lines

In [5]:
def read_report():
    with open("human-activity-smart-devices/report.csv") as f:
        csv_reader = csv.reader(f)
        lines = []
        for i, row in enumerate(csv_reader):
            lines.append(row)
        list_activities = []
        for line in lines[1:]:
            list_activities.append(Activity(line[1], line[3], line[4]))

    return list_activities

In [6]:
def get_time(string):
    split1 = string.split(" ")
    ymd = split1[0].split("-")
    if len(ymd) < 2:
        ymd = split1[0].split("/")
    hms = split1[1].split(":")
    if len(hms) > 2:
        sm = hms[2].split(".")
    else:
        sm = ["0", "0"]
    return datetime.datetime(int(ymd[0]), int(ymd[1]), int(ymd[2]), int(hms[0]), int(hms[1]), int(sm[0]), int(float("0." + sm[1])*1000000))

In [7]:
def experiment(data, columns, y_data):
    df = pd.DataFrame(data, columns=columns)
    ydf = pd.DataFrame(y_data)
    x_train, x_test = df[:4698], df[4698:]
    y_train, y_test = ydf[:4698], ydf[4698:]
    scaler = preprocessing.StandardScaler()
    names = x_train.columns
    scaled_df = scaler.fit_transform(x_train)
    x_train = pd.DataFrame(scaled_df, columns=names)

    names = x_test.columns
    scaled_df = scaler.fit_transform(x_test)
    x_test = pd.DataFrame(scaled_df, columns=names)
    x_test = np.expand_dims(x_test, axis=2)
    x_train = np.expand_dims(x_train, axis=2)


    model = Sequential()
    model.add(Conv1D(filters=10, kernel_size=2, padding='same', activation='relu', input_shape=(10,1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(8, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    model.fit(x_train, y_train, epochs=3)

    scores = model.evaluate(x_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1] * 100))

    y_pred_keras = model.predict(x_test)
    y_test = y_test.values

    plt.figure(figsize=(10, 10))
    plt.plot([0, 1], [0, 1], 'k--')

    list_of_keys = list(activities.keys())
    print(list_of_keys)
    for i in range(8):
        print(y_test[:, i])
        print(y_pred_keras.shape)
        fpr, tpr, threshold = roc_curve(y_test[:, i], y_pred_keras[:, i])  # YOUR CODE HERE construct ROC Curve
        plt.plot(fpr, tpr, label='{}, AUC = {:.3f}'.format(list_of_keys[i], auc(fpr, tpr)))


    # fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)
    # auc_keras = auc(fpr_keras, tpr_keras)
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend()
    plt.show()

In [ ]:
acts = read_report()
columns, data = make_data()

complete_lines = []
for line in data:
    if "-" not in line:
        complete_lines.append(line)
final_data = []


for line in complete_lines:
    for activity in acts:
        if activity.in_between(line[0]):
            final_data.append(line + [activity.get_class_number()])
            break

final_data = sorted(final_data, key=lambda x: x[0])
mid_data = []
for f in final_data:
    mid_data.append(f[1:])
float_data = []
y_data = []
for f in mid_data:
    float_data.append([float(x) for x in f[:-1]])
    y_data.append(f[-1])
experiment(float_data, columns[1:], y_data)